# Amazon Review Data Recommendation System

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

### Read the data file

In [3]:
import json

data = []
with open('/Users/seungwooseo/Desktop/Python/Amazon_Review_Kaggle/Cell_Phones_and_Accessories_5.json', 'r') as f:
    for line in f:
        data.append(json.loads(line))

df = pd.DataFrame(data)


In [4]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't li...,4.0,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they ...,5.0,Really great product.,1389657600,"01 14, 2014"
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",These are awesome and make my phone look so st...,5.0,LOVE LOVE LOVE,1403740800,"06 26, 2014"
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",Item arrived in great time and was in perfect ...,4.0,Cute!,1382313600,"10 21, 2013"
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]","awesome! stays on, and looks great. can be use...",5.0,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013"


### Exploratory Data Analysis

In [5]:
# Info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194439 entries, 0 to 194438
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   reviewerID      194439 non-null  object 
 1   asin            194439 non-null  object 
 2   reviewerName    190920 non-null  object 
 3   helpful         194439 non-null  object 
 4   reviewText      194439 non-null  object 
 5   overall         194439 non-null  float64
 6   summary         194439 non-null  object 
 7   unixReviewTime  194439 non-null  int64  
 8   reviewTime      194439 non-null  object 
dtypes: float64(1), int64(1), object(7)
memory usage: 13.4+ MB


In [6]:
# Describe
df.describe()

,overall,unixReviewTime
count,194439.000000,1.944390e+05
mean,4.129912,1.368714e+09
std,1.222499,3.230032e+07
min,1.000000,9.828000e+08
25%,4.000000,1.357603e+09
50%,5.000000,1.374538e+09
75%,5.000000,1.390262e+09
max,5.000000,1.406074e+09


In [7]:
df.columns

Index(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText',
       'overall', 'summary', 'unixReviewTime', 'reviewTime'],
      dtype='object')

In [8]:
# Average Number of Buying Per Person
round(df['reviewerID'].value_counts().mean())

7

In [9]:
# Average Number of Selling Per Product
round(df['asin'].value_counts().mean())

19

In [10]:
# Number of Selling by Product
df['asin'].value_counts()

asin
B005SUHPO6    837
B0042FV2SI    694
B008OHNZI0    657
B009RXU59C    636
B000S5Q9CA    628
             ... 
B006W95VHU      5
B004T6S7C4      5
B009P5XSAU      5
B004TETBQ2      5
B0089ZUTWM      5
Name: count, Length: 10429, dtype: int64

In [11]:
# Average Overall of Each Product
asin_overall = df.groupby(by='asin')['overall'].mean()
asin_overall = asin_overall.reset_index()
asin_overall

,asin,overall
0,120401325X,4.428571
1,3998899561,4.600000
2,6073894996,3.891892
3,7532385086,4.444444
4,7887421268,4.000000
...,...,...
10424,B00LH1QZR8,5.000000
10425,B00LH1R3C4,5.000000
10426,B00LH1R5HW,5.000000
10427,B00LH1R6SK,5.000000


In [12]:
# Average of All overall
df['overall'].mean()

4.129912208970422

In [13]:
# Top 10 Purchase Products
asin_top = df['asin'].value_counts().sort_values(ascending=False).head(10)
asin_top = pd.DataFrame(asin_top).reset_index()
asin_top

,asin,count
0,B005SUHPO6,837
1,B0042FV2SI,694
2,B008OHNZI0,657
3,B009RXU59C,636
4,B000S5Q9CA,628
5,B007FHX9OK,557
6,B0073FCPSK,526
7,B008DJIIG8,510
8,B0088LYCZC,488
9,B0015RB39O,466


In [14]:
for product in asin_top['asin']:
    asin_top.loc[asin_top['asin']==product, 'overall'] = asin_overall.loc[asin_overall['asin']==product, 'overall'].values[0]

In [15]:
asin_top

,asin,count,overall
0,B005SUHPO6,837,4.367981
1,B0042FV2SI,694,3.974063
2,B008OHNZI0,657,4.430746
3,B009RXU59C,636,4.676101
4,B000S5Q9CA,628,4.507962
5,B007FHX9OK,557,4.482944
6,B0073FCPSK,526,4.349810
7,B008DJIIG8,510,4.472549
8,B0088LYCZC,488,4.092213
9,B0015RB39O,466,3.628755


In [1]:
import seaborn as sns
import chart_studio.plotly as py
import cufflinks as cf
import plotly.express as px
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objects as go
import plotly
init_notebook_mode(connected=True)
cf.go_offline()

In [35]:
fig = go.Figure()
fig.add_trace(go.Bar(x=asin_top['asin'], y=asin_top['overall'], hovertemplate="ASIN=%{x}<br>Overall=%{y}"))
fig.update_layout(title='Overall of top 10 products',title_x=0.5)

## Collaborative Filtering

### Cosine Similarity

#### Labelling each ReviewerID and Asin

In [36]:
#import label encoder
from sklearn.preprocessing import LabelEncoder

In [37]:
le = LabelEncoder()

In [38]:
le.fit(df['reviewerID'])

LabelEncoder()

In [39]:
origin_df = pd.DataFrame()
origin_df['user_id'] = le.transform(df['reviewerID'])

In [40]:
le.fit(df['asin'])

LabelEncoder()

In [41]:
origin_df['item_id'] = le.transform(df['asin'])

In [42]:
origin_df

,user_id,item_id
0,14885,0
1,26539,0
2,13358,0
3,27221,0
4,26730,0
...,...,...
194434,7020,10428
194435,1229,10428
194436,18730,10428
194437,4784,10428


In [43]:
#Mother Matrix for calculation of similarity
origin_df = pd.concat([origin_df,df['overall']], axis=1)

In [44]:
#Make a table for iterrow works
origin_df

,user_id,item_id,overall
0,14885,0,4.0
1,26539,0,5.0
2,13358,0,5.0
3,27221,0,4.0
4,26730,0,5.0
...,...,...,...
194434,7020,10428,5.0
194435,1229,10428,5.0
194436,18730,10428,5.0
194437,4784,10428,5.0


#### Divide them into train_split and test_split & Calculate the similarity

In [45]:
# Train_test_split
from sklearn.model_selection import train_test_split

In [46]:
# Split the database into train and test
train_data, test_data = train_test_split(origin_df, test_size=0.3, random_state=33)

In [47]:
train_data

,user_id,item_id,overall
97096,9057,5184,5.0
125085,23878,6380,5.0
138185,27552,7055,4.0
4438,26163,316,5.0
139720,22712,7140,1.0
...,...,...,...
75971,26565,4225,5.0
158921,25171,8146,5.0
131650,15045,6691,4.0
104152,6232,5476,5.0


In [49]:
# Count the whole rows and columns to make a similarity table
n_users = origin_df['user_id'].nunique()
n_items = origin_df['item_id'].nunique()
print('Number of Users: ', n_users)
print('Number of Items: ', n_items)

Number of Users:  27879
Number of Items:  10429


In [50]:
#Make a similarity table
train_data_matrix = np.zeros([n_users,n_items])
test_data_matrix = np.zeros([n_users,n_items])

In [51]:
# Composing a train_data_matrix
for train in train_data.itertuples():
    train_data_matrix[train[1], train[2]] = train[3] #train[0] is index

# Composing a test_data_matrix
for test in test_data.itertuples():
    test_data_matrix[[test[1]-1, test[2]-1]] = test[3] #test[0] is index

In [59]:
train_data_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

#### Calculate the similarity

In [60]:
from sklearn.metrics.pairwise import pairwise_distances
user_similarity = pairwise_distances(train_data_matrix, metric='cosine')
item_similarity = pairwise_distances(train_data_matrix.T, metric='cosine')

In [61]:
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        #You use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis]) 
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])     
    return pred

In [62]:
item_prediction = predict(train_data_matrix, item_similarity, type='item')
user_prediction = predict(train_data_matrix, user_similarity, type='user')

In [63]:
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(prediction, ground_truth):#Filtering the comparable values
    prediction = prediction[ground_truth.nonzero()].flatten() 
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))

In [64]:
print('User-based CF RMSE: ' + str(rmse(user_prediction, test_data_matrix)))
print('Item-based CF RMSE: ' + str(rmse(item_prediction, test_data_matrix)))

User-based CF RMSE: 4.295441556611824
Item-based CF RMSE: 4.295440567886835
